In [19]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

In [20]:
train_features_data=pd.read_csv('train_features.csv')
train_labels_data=pd.read_csv('train_labels.csv')
test_features_data=pd.read_csv('test_features.csv')
# add labels with matching pid to features
labels_copied = pd.DataFrame()
labels_copied = train_labels_data.loc[train_labels_data.index.repeat(12)]
labels_copied = labels_copied.drop(columns=['pid'])
labels = train_labels_data.drop(columns=['pid'])
LABELS1 = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
LABELS2 = ['LABEL_Sepsis']
LABELS3 = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

## Subtask 1

In [21]:
X = train_features_data.drop(columns=['pid', 'Time']).reset_index(drop=True).fillna(test_features_data.mean())
y = labels_copied[LABELS1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, shuffle=True)

TEST_X = test_features_data.drop(columns=['pid', 'Time']).reset_index(drop=True).fillna(test_features_data.mean())

### RandomForest model and Roc_Auc_Score using every hour per patient as sample for training

In [22]:
clf=RandomForestClassifier(n_estimators=300, class_weight=None, n_jobs=-1 )
clf.fit(X_train, y_train)
y_pred_proba=clf.predict_proba(X_test)
err=np.empty(10)
list_proba=list()
for i,label in enumerate(LABELS1):
    err[i]=roc_auc_score(y_test[label], y_pred_proba[i][:,1])
    list_proba.append(y_pred_proba[i][:,1])
display(err)
display('Average score: ', np.mean(err))

array([0.82584325, 0.71327951, 0.73359083, 0.73875457, 0.73827658,
       0.76326581, 0.80074495, 0.77722549, 0.82695949, 0.92912107])

'Average score: '

0.7847061548709628

### Predictions for Test_features

In [23]:
TEST_y_pred_proba=clf.predict_proba(TEST_X)

### Reduce values by 12 for TEST_y_pred_proba

In [24]:
TEST_list_proba=list()
for j in range(10):
    TEST_y_pred_proba_reduced=np.empty(int(len(TEST_y_pred_proba[j][:,1])/12))
    counter=0
    for splits in np.split(np.array(TEST_y_pred_proba[j][:,1]), int(len(TEST_y_pred_proba[j][:,1])/12)):
        TEST_y_pred_proba_reduced[counter]=splits.mean() 
        counter=counter+1
    TEST_list_proba.append(TEST_y_pred_proba_reduced)
proba_subtask1=TEST_list_proba

## Subtask 2

In [25]:
y = labels_copied[LABELS2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, shuffle=True)

### RandomForest model and Roc_Auc_Score using every hour per patient as sample

In [26]:
clf=RandomForestClassifier(n_estimators=300, class_weight=None, n_jobs=-1)

clf.fit(X_train, y_train)
y_pred_proba=clf.predict_proba(X_test)
err=roc_auc_score(y_test, y_pred_proba[:,1])
display(err)

C:\Users\Manue\anaconda3\envs\ML\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.8025628978270413

### Prediction for Test features

In [27]:
TEST_y_pred_proba=clf.predict_proba(TEST_X)

### Reduce values by 12 for TEST_y_pred_proba

In [28]:
TEST_y_pred_proba_reduced=np.empty(int(len(TEST_y_pred_proba[:,1])/12))
counter=0
for splits in np.split(np.array(TEST_y_pred_proba[:,1]), int(len(TEST_y_pred_proba[:,1])/12)):
    TEST_y_pred_proba_reduced[counter]=splits.mean() 
    counter=counter+1
proba_subtask2=TEST_y_pred_proba_reduced

## Subtask 3

In [29]:
y = labels_copied[LABELS3]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, shuffle=True, random_state=7)

### Implement MultiTaskLassoCV model

In [30]:
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error


cv = RepeatedKFold(n_splits=10, n_repeats=3)

model = RidgeCV(alphas=(0.1, 1, 10), cv=cv)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print(mean_squared_error(y_test, y_pred))

59.14867326260958


### Predictions for Test features

In [31]:
TEST_y_pred_values=model.predict(TEST_X)
display(TEST_y_pred_values)

array([[18.42622816, 84.66780738, 97.16717296, 85.20206793],
       [16.55886895, 80.82206443, 98.07685707, 80.63078246],
       [16.36246332, 83.15365173, 98.0393793 , 83.01494522],
       ...,
       [18.14952688, 77.00544142, 97.73800189, 87.66538068],
       [18.44739023, 79.02549924, 97.74487106, 87.84955082],
       [17.45553031, 77.13355053, 97.75610598, 86.55506884]])

### Reduce values by 12 for TEST_y_pred_values

In [32]:
TEST_list_values=list()
for j in range(4):
    TEST_y_pred_values_reduced=np.empty(int(len(TEST_y_pred_values[:,j])/12))
    counter=0
    for splits in np.split(np.array(TEST_y_pred_values[:,j]), int(len(TEST_y_pred_values[:,j])/12)):
        TEST_y_pred_values_reduced[counter]=splits.mean() 
        counter=counter+1
    TEST_list_values.append(TEST_y_pred_values_reduced)
proba_subtask3=TEST_list_values

### Write values into sample dataframe

In [33]:
filename = 'sample.zip'
df_submission = pd.read_csv(filename)
for i,label in enumerate(LABELS1):
    # round classification labels
    df_submission[label]=proba_subtask1[i]
df_submission[LABELS2[0]]=proba_subtask2
for i,label in enumerate(LABELS3):
    # round classification labels
    df_submission[label]=proba_subtask3[i]
df_submission.to_csv('submission.csv',index=False)
df_submission.to_csv('prediction.zip', index=False, float_format='%.3f', compression='zip')